# Simple Proof of Work Blockchain example

   

In [2]:
from ethereum import utils
import time, sys



We first start by defining what a block in the blockchain. In this example, we are highly simplifying all the concepts

Simply put, a blockchain block has at least 2 important data structures whithin it, a list of transactions (commonly know as the body) and a block header.

The block header has important metadata, namely it has a reference to the last block (hence the linked list aspect of blockchain), it keeps his position number on the linked list and has the nonce solution from the proof of work.


In [4]:
class Block:
  def __init__(self, timestamp, transactions, previous_hash, nonce, block_number):
    # Block headers
    self.header = {}
    self.header["timestamp"] = timestamp
    self.header["previous_hash"] = previous_hash
    self.header["nonce"] = nonce
    self.header["block_number"] = block_number
    self.transactions = transactions
    self.header["hash"] = self.hash_block()


  

Further we define a simplyfied hashing function that simply hashes the concatenation of the different elecemtns of the block header along with the transaction list

In [6]:
  # Simplified hashing, concatenate the headers and the data and hash
  def hash_block(self):
    return utils.encode_hex(utils.sha3(str(self.header["timestamp"]) + str(self.header["previous_hash"]) + str(self.transactions) ))

Next we define a global function that creates a new block

In [7]:
def create_block(txns, previous_hash, nonce, block_number):
    # return Block(1525755750, data, previous_hash, nonce)
    return Block(time.time().__int__(), txns, previous_hash, nonce, block_number)


A blockchain is a simple append only linked list of blocks with specific rules as to how to add blocks to it. THe first block is called the genesis block, and is usually where certain accounts are initialized


In [9]:
class Blockchain:
    def __init__(self):
        genesis = create_block(["Genesis"], utils.encode_hex(utils.sha3("")), 0, 0)
        self.blocks = [genesis]
    
    # Function used to append to blocks
    def append_block(self, txns):
        previous_block = self.blocks[len(self.blocks) - 1]
        # We create a new candidate block
        new_block = create_block(txns, previous_block.header["hash"], 0, len(self.blocks))
        # Find its proof of work solution
        nonce, winning_hash = ProofOfWork(new_block).find_solution()
        # update the fields
        new_block.header["nonce"] = nonce
        new_block.header["hash"] = winning_hash
        # and append to the blockchain
        return self.blocks.append(new_block)
    
    # Function used to get a block by its hash
    def getBlockByHash(self, target_hash):
        for i in range(len(self.blocks) -1, 0, -1):
            if self.blocks[i].header["hash"] == target_hash:
                return self.blocks[i]
        return "Block not found"

To add a block to the blockchain when using the proof of work consensus algorithm, one needs find a nonce that upon hashing the whole block along with the nonce, finds a solution that is under a certain difficulty

In [10]:
class ProofOfWork:
    def __init__(self, current_block):
        target_bits = 12 # can be changed
        self.current_block = current_block
        self.difficulty =  1 << 256-target_bits

    def create_pow_hash(self, nonce):
        return  utils.encode_hex(utils.sha3(str(self.current_block.header["previous_hash"]) + str(self.current_block.transactions) + str(self.current_block.header["timestamp"]) + str(nonce)))

    def find_solution(self):
        nonce = 0
        while(True):
            pow_hash = self.create_pow_hash(nonce)
            hash_int = utils.parse_int_or_hex("0x"+pow_hash)
            if hash_int < self.difficulty:
                return nonce, pow_hash
            nonce += 1
        return 0, 0



Now we need a way to validate a block, ensure that the hash of the block is less than the difficulty

In [11]:
def validate_block(block):
    pow = ProofOfWork(block)
    pow_hash = pow.create_pow_hash(pow.current_block.header["nonce"])
    hash_int = utils.parse_int_or_hex("0x"+pow_hash)
    return hash_int < pow.difficulty

In [ ]:
We initialize the blockchain, it should have our hard coded genesis block

In [14]:
bc = Blockchain()
print(bc.blocks[0].header)
print(bc.blocks[0].transactions)

{'nonce': 0, 'timestamp': 1526521364, 'previous_hash': 'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470', 'hash': 'c3d57bcfd0fe249279633fea1140d2c14343bffe3c93e3122846e225a82d2e1d', 'block_number': 0}
['Genesis']


Let's add a block to it

In [15]:
bc.append_block(["hello"])
# print statements...
print(bc.blocks[1].header)
print(bc.blocks[1].transactions)

{'nonce': 524, 'timestamp': 1526521398, 'previous_hash': 'c3d57bcfd0fe249279633fea1140d2c14343bffe3c93e3122846e225a82d2e1d', 'hash': '000e380639a42d25d41e22a294cbf9cfc36fcd9916a724ce810f7589b7895f7e', 'block_number': 1}
['hello']


We add more, and validate that they satisfy the proof of work requirement

In [16]:
bc.append_block(["bonjour"])
print(bc.blocks[2].header)
bc.append_block(["hola"])
print(bc.blocks[3].header)
print(validate_block(bc.blocks[1]))

{'nonce': 3263, 'timestamp': 1526521445, 'previous_hash': '000e380639a42d25d41e22a294cbf9cfc36fcd9916a724ce810f7589b7895f7e', 'hash': '00081e027a03a1b2c12df594dd701245da974ee95699448168850ac70cc46b08', 'block_number': 2}
{'nonce': 1395, 'timestamp': 1526521445, 'previous_hash': '00081e027a03a1b2c12df594dd701245da974ee95699448168850ac70cc46b08', 'hash': '000140b78982c9d0b4216a9cb99d9cd365c2a43a355b676811ff030b4c24abba', 'block_number': 3}
True


Now, we have a blockchain, and we can search through it by going through every block until we find the number that we want

In [18]:
search_block = (bc.getBlockByHash(bc.blocks[1].header["hash"]))
print(search_block.header)

{'nonce': 524, 'timestamp': 1526521398, 'previous_hash': 'c3d57bcfd0fe249279633fea1140d2c14343bffe3c93e3122846e225a82d2e1d', 'hash': '000e380639a42d25d41e22a294cbf9cfc36fcd9916a724ce810f7589b7895f7e', 'block_number': 1}
